In [1]:
# home_directory = ''
home_directory = 'drive/My Drive/Colab Notebooks/poe/'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.4MB/s 
     |████████████████████████████████| 890kB 38.8MB/s 
     |████████████████████████████████| 1.1MB 37.1MB/s 
     |████████████████████████████████| 2.9MB 40.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a85e007accd1b2afbe039e6a8ae3c47a56982609e01d0a26538dc359fcf9d5d9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Import Libraries & Load Data

In [4]:
import numpy as np
import pandas as pd 

import os
import re
import string
import random
import time
import datetime

from collections import Counter
import itertools
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
plt.style.use('bmh')

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

pd.set_option('display.max_rows', 100)

In [5]:
story_df = pd.read_csv(home_directory + 'poe_short_stories.csv')

# Clean Short Story Data

In [9]:
story_df[story_df.classification=='Essay']

,title,text,wikipedia_title,publication_date,first_published_in,classification,notes,normalized_date
62,* PHILOSOPHY OF FURNITURE,"In the internal decoration, if not in the exte...",?,May 1840,?,Essay,?,May 1840
63,MAELZEL’S CHESS-PLAYER,PERHAPS no exhibition of the kind has ever eli...,?,April 1836,?,Essay,?,April 1836
64,OLD ENGLISH POETRY,IT should not be doubted that at least one-thi...,?,?,?,Essay,?,?
67,THE POETIC PRINCIPLE,"IN speaking of the Poetic Principle, I have no...",?,"August 17, 1849",?,Essay,?,August 1849


In [10]:
# remove essays (non-fiction)
story_df = story_df.loc[story_df['classification'] != 'Essay'].reset_index()

In [12]:
story_df[story_df.normalized_date == '? 1843']

,index,title,text,wikipedia_title,publication_date,first_published_in,classification,notes,normalized_date
48,48,THE PIT AND THE PENDULUM,Impia tortorum longos hic turba furores Sangui...,The Pit and the Pendulum,? 1843,The Gift: A Christmas and New Year's Present,Horror,?,? 1843


"Annual issues were published in the fall of the preceding year.  The month for publication of this issue is thought to be October due to a reprint of "The Pit and the Pendulum" in the October 22, 1842 New York Spectator"

Source:  http://www.isfdb.org/cgi-bin/pl.cgi?296127

In [13]:
story_df.loc[story_df['title'] == 'THE PIT AND THE PENDULUM', 'normalized_date'] = 'October 1842'

In [14]:
story_df[story_df.normalized_date == '? 1841']

,index,title,text,wikipedia_title,publication_date,first_published_in,classification,notes,normalized_date
6,6,ELEONORA,Sub conservatione formae specificae salva anim...,Eleonora,? 1841,The Gift for 1842,Romance,?,? 1841


Since Eleonara appeared in the same publication (The Gift) as The Pit and the Pendulum, we can assume it had a similar date of publication (October of the previous year)

In [15]:
story_df.loc[story_df['title'] == 'ELEONORA', 'normalized_date'] = 'October 1841'

# Text Generation - GPT-2
## Process Text and Create Dataset

http://jalammar.github.io/illustrated-gpt2/
https://medium.com/@stasinopoulos.dimitrios/a-beginners-guide-to-training-and-generating-text-using-gpt2-c2f2e1fbd10a
https://huggingface.co/transformers/model_doc/gpt2.html
https://towardsdatascience.com/step-by-step-guide-on-how-to-train-gpt-2-on-books-using-google-colab-b3c6fa15fef0
https://medium.com/swlh/fine-tuning-gpt-2-for-magic-the-gathering-flavour-text-generation-3bafd0f9bb93
https://colab.research.google.com/drive/16UTbQOhspQOF3XlxDFyI28S-0nAkTzk_#scrollTo=v4XhewaV93-_

In [46]:
RANDOM_SEED = 73
BATCH_SIZE = 4

EPOCHS = 4
SAMPLE_EVERY = 100

MAX_SEQUENCE_LENGTH = 512

In [47]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [48]:
max_story_length = max([len(tokenizer.encode(story)) for story in story_df['text'].values])
min_story_length = min([len(tokenizer.encode(story)) for story in story_df['text'].values])

Token indices sequence length is longer than the specified maximum sequence length for this model (9052 > 1024). Running this sequence through the model will result in indexing errors


In [49]:
print('Longest Edgar Allen Poe Short Story:', max_story_length, 'tokens long.')
print('Shortest Edgar Allen Poe Short Story:', min_story_length, 'tokens long.')

Longest Edgar Allen Poe Short Story: 25411 tokens long.
Shortest Edgar Allen Poe Short Story: 1305 tokens long.


In [54]:
class PoeStoryDataset(Dataset):
    
    def __init__(self, data, tokenizer, gpt2_type='gpt2', max_length=MAX_SEQUENCE_LENGTH):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        
        for i in data:
            for j in split_into_sentences(i):
            
                encodings_dict = tokenizer('<BOS>' + j + '<EOS>',
                                           truncation=True,
                                           max_length=max_length,
                                           padding='max_length'
                                          )

                self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
                self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        
        return self.input_ids[idx], self.attn_masks[idx]
        

In [56]:
story_dataset = PoeStoryDataset(story_df['text'].values, tokenizer, max_length=MAX_SEQUENCE_LENGTH)

### Train/Validation Split

In [57]:
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size

In [58]:
story_train_size, story_val_size = train_val_split(0.8, story_dataset)

# random split imported from troch.utils
story_train_dataset, story_val_dataset = random_split(story_dataset, [story_train_size, story_val_size])

### Apply Random Seeds

In [59]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

### Instantiate DataLoaders and Define Model Creation Function

---



In [60]:
def create_dataloaders(train_dataset, val_dataset, bs):
    train_dataloader = DataLoader(train_dataset,
                                  sampler=RandomSampler(train_dataset),
                                  batch_size=bs)

    val_dataloader = DataLoader(val_dataset,
                                sampler=SequentialSampler(val_dataset),
                                batch_size=bs)
    
    return train_dataloader, val_dataloader

In [61]:
story_train_dataloader, story_val_dataloader = create_dataloaders(story_train_dataset, story_val_dataset, BATCH_SIZE)

In [62]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_SEQUENCE_LENGTH).from_pretrained('gpt2', output_hidden_states=True)

In [63]:
# helper function for logging time
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [64]:
# hyperparameters

learning_rate = 5e-4
eps = 1e-8
warmup_steps = 1e2

In [65]:
# create text generation seed prompt
device = torch.device('cuda')
# device = torch.device('cpu')
prompt = "<BOS>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

In [66]:
def create_model(train_dataloader, val_dataloader, file_name):

    model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
    model.resize_token_embeddings(len(tokenizer))
    
    model.cuda()
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)

    total_steps = len(train_dataloader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=warmup_steps,
                                                num_training_steps=total_steps)
    
    total_t0 = time.time()
    model = model.to(device)

    for epoch_i in range(0, EPOCHS):

        print(f'Epoch {epoch_i + 1} of {EPOCHS}')

        t0 = time.time()
        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):

            b_input_ids = batch[0].to(device)
            b_labels = batch[0].to(device)
            b_masks = batch[1].to(device)

            model.zero_grad()        

            outputs = model(b_input_ids,
                            labels=b_labels, 
                            attention_mask=b_masks,
                            token_type_ids=None)

            loss = outputs[0]  

            batch_loss = loss.item()
            total_train_loss += batch_loss

            if step % SAMPLE_EVERY == 0 and step != 0:
                
                model.eval()
                sample_outputs = model.generate(
                                        generated,
                                        do_sample=True,   
                                        top_k=50, 
                                        max_length=200,
                                        top_p=0.95, 
                                        num_return_sequences=1
                                    )
                for i, sample_output in enumerate(sample_outputs):
                      print(f'Example output: {tokenizer.decode(sample_output, skip_special_tokens=True)}')

                model.train()

            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)       
        training_time = format_time(time.time() - t0)

        print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

        t0 = time.time()

        print('Evaluating Model')

        model.eval()

        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in val_dataloader:
            b_input_ids = batch[0].to(device)
            b_labels = batch[0].to(device)
            b_masks = batch[1].to(device)

            with torch.no_grad():        

                outputs  = model(b_input_ids,  
                                 attention_mask=b_masks,
                                 labels=b_labels)

                loss = outputs[0]  

            batch_loss = loss.item()
            total_eval_loss += batch_loss        

        avg_val_loss = total_eval_loss / len(val_dataloader)

        validation_time = format_time(time.time() - t0)    

        print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')

    print(f'Total training took {format_time(time.time()-total_t0)}')

    torch.save(model.state_dict(), home_directory + file_name)
    return model

# Create Story Model

In [71]:
story_model = create_model(story_train_dataloader, story_val_dataloader, 'story_model_4_epoch.pth')


Epoch 1 of 4


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Batch 100 of 3044. Loss:0.11765287816524506. Time:0:02:01
Example output:  and all that I have seen, was just my way of getting to the point whereI was sure of nothing.


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Batch 200 of 3044. Loss:0.5541245937347412. Time:0:04:07
Example output: There is no doubt that the present state of our society could not possibly have been induced by the influence of a


...

Batch 2800 of 3044. Loss:0.08007431030273438. Time:0:58:21
Example output: The result was that the bandage crossed my bosom in an unnatural way.


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Batch 2900 of 3044. Loss:0.11757238209247589. Time:1:00:26
Example output: —who, indeed, could he be dreaming—it is impossible to say—for what could he possibly dreaming, would have been dreaming on its own?


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Batch 3000 of 3044. Loss:0.10439801216125488. Time:1:02:31
Example output: It was of a dull grayish-white, the color of the earth.
Average Training Loss: 0.1075666684849474. Epoch time: 1:03:28
Evaluating Model
Validation loss: 0.24512592048978243. Validation Time: 0:05:11
Total training took 4:34:57


# Generate Story Lines

In [72]:
story_model.eval()

sample_outputs = story_model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length=512,
                                top_p=0.95, 
                                num_return_sequences=100
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: He would not—he would not—he would not withdraw his claim to our friendship—to the city of London—would insist upon a little personal foot-bail, after the placing his claim upon some trifling account with a firm and fixed footman, whom he could always catch by the wool.


1: This functionary, without much ill-breeding, was of an unassuming and petulant character, and I was sure that he employed it, in a rational way, as if the only natural and convenient vehicle for the exercise of the poetic sentiment.


2: ”     “Well, I have no fever to speak, Mounseer Frog, we will take care of you until night, and then, we will take care of you until you return home”.


3: There was an air of complacent complaisance pervading every portion of his countenance, and his eye-glass looked fixedly and fixedly forever upon the ceiling, as if the rays of the city upon the earth, and forever upon the moon.


4: —must I say?


5: You will notice that we have a wild luxuriant influence upon all heavenly o